# Introduction
In this notebook, you will learn how Myscale can assist you in creating a question-answering application with openai api. There are three primary components required to construct a question-answering system:
1. A vector index for semantic search storage and execution.
2. A retriever model to embed contextual passages.
3. OpenAI API for answer extraction.

We will use bitcoin_articles dataset. The dataset contains a collection of news articles on Bitcoin that have been obtained through web scraping from different sources on the Internet using the Newscatcher API. We'll use the retriever to create embeddings for the context passages, index them in the vector database, and execute a semantic search to retrieve the top k most relevant contexts with potential answers to our question. OpenAI API will then be used to generate answers based on the returned contexts.

### Zero-shot GPT-3.5
As comparision, we first implement zero-shot QA with GPT-3.5, We can see that this answer is relatively simple.

In [20]:
import sys
import os
import openai
sys.argv=['']
openai.api_key  = ''
question = "what is the difference between bitcoin and traditional money?"
print("Example: Zero-Shot")
print('Q: ',question)
response = openai.Completion.create(
          engine='gpt-3.5-turbo',
          prompt=question,
          max_tokens=4000,
          temperature=0,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=["\n"]
        )
print('A: ',response["choices"][0]["text"])

Example: Zero-Shot
Q:  what is the difference between bitcoin and traditional money?
A:  Bitcoin is decentralized and operates on a blockchain, while traditional money is centralized and controlled by governments and financial institutions.


### Load data and Prepocess
We first establish a small database for retrieval, after loading data we perform basic data manipulation tasks such as eliminating duplicate entries and removing empty cells. 

In [21]:
import pandas as pd

data_raw = pd.read_csv('/root/xuying_experiments/bitcoin_articles.csv')
data_raw.head(3)

,article_id,title,author,published_date,link,clean_url,excerpt,summary,rights,article_rank,topic,country,language,authors,media,twitter_account,article_score
0,57a00c1140cbd3af79e77bf0e4e6af48,62% of Bitcoin Has Not Moved in a Year as Long...,Jamie McNeill,04-10-2022 17:15,https://www.business2community.com/crypto-news...,business2community.com,"Over the course of the last few years, there h...","Over the course of the last few years, there h...",business2community.com,1595,finance,US,en,Jamie McNeill,https://www.business2community.com/wp-content/...,@Jamie_DeFi,8.556426
1,21b48b3731c03466be3fac4be6c7dc67,The Orange Party Issue Playlist,Bitcoin Magazine,05-10-2022 21:17,https://bitcoinmagazine.com/culture/orange-par...,bitcoinmagazine.com,News Links: Russia Legalizing Bitcoin And Cryp...,Russia Legalizing Bitcoin And Crypto Is A Matt...,bitcoinmagazine.com,6284,news,US,en,Bitcoin Magazine,https://bitcoinmagazine.com/.image/t_share/MTk...,NaN,8.507881
2,77030740ee160ad68c25e4e63515dd77,How Many Bitcoins Are There?,AOL Staff,04-10-2022 21:44,https://www.gobankingrates.com/investing/crypt...,gobankingrates.com,Bitcoin has a maximum supply of 21 million. Ho...,Bitcoin has a maximum supply of 21 million. Ho...,aol.com,5044,news,US,en,"AOL Staff,David Granahan",https://s.yimg.com/ny/api/res/1.2/wPK4V8gjwjrD...,@AOL,8.483973


In [22]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   article_id       2500 non-null   object 
 1   title            2500 non-null   object 
 2   author           2264 non-null   object 
 3   published_date   2500 non-null   object 
 4   link             2500 non-null   object 
 5   clean_url        2500 non-null   object 
 6   excerpt          2468 non-null   object 
 7   summary          2473 non-null   object 
 8   rights           2468 non-null   object 
 9   article_rank     2500 non-null   int64  
 10  topic            2500 non-null   object 
 11  country          2500 non-null   object 
 12  language         2500 non-null   object 
 13  authors          2264 non-null   object 
 14  media            2461 non-null   object 
 15  twitter_account  1968 non-null   object 
 16  article_score    2500 non-null   float64
dtypes: float64(1),

In [23]:
data_raw.drop_duplicates(subset=['summary'], keep='first', inplace=True)
data_raw.dropna(subset=['summary'],inplace=True)

In [24]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1931 entries, 0 to 2499
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   article_id       1931 non-null   object 
 1   title            1931 non-null   object 
 2   author           1731 non-null   object 
 3   published_date   1931 non-null   object 
 4   link             1931 non-null   object 
 5   clean_url        1931 non-null   object 
 6   excerpt          1906 non-null   object 
 7   summary          1931 non-null   object 
 8   rights           1906 non-null   object 
 9   article_rank     1931 non-null   int64  
 10  topic            1931 non-null   object 
 11  country          1931 non-null   object 
 12  language         1931 non-null   object 
 13  authors          1731 non-null   object 
 14  media            1900 non-null   object 
 15  twitter_account  1450 non-null   object 
 16  article_score    1931 non-null   float64
dtypes: float64(1),

### Load model to embed data
we will have to initiate our retriever, which will primarily perform two tasks:

1. Produce embeddings for each context passage (context vectors/embeddings)
2. Produce an embedding for our queries (query vector/embedding)

In [25]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [26]:
from tqdm.auto import tqdm

# we will use batches of 1
batch_size = 1
res = data_raw['summary'].values.tolist()
res_feature = []

for i in tqdm(range(0, len(res), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(res))
    
    # extract batch
    batch = res[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()[0]
    # get metadata
    # meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    res_feature.append(emb)

  0%|          | 0/1931 [00:00<?, ?it/s]

In [27]:
from pandas.core.frame import DataFrame
data_raw['summary_feature'] = res_feature

In [28]:
data = data_raw[['article_id','title','author','link','summary','article_rank','summary_feature']]
data.to_csv('bitcon.csv')

### Create table in MyScale

In [12]:
import clickhouse_connect as client

# initialize MyScale client
client.get_client(host='YOUR_CLUSTER_HOST', port=8443, username='YOUR_USERNAME', password='YOUR_CLUSTER_PASSWORD')



In [32]:
# create table for IMF texts
client.command("""
CREATE TABLE default.LLM_bitcoin_demo_qa_1
(
    id UInt64,
    article_id String,
    title String,
    author String,
    link String,
    summary String,
    article_rank UInt64,
    summary_feature Array(Float32),
    CONSTRAINT vector_len CHECK length(summary_feature) = 384
)
ENGINE = MergeTree ORDER BY id
""")

### Uploading data

In [37]:
batch_size = 100
for i in tqdm(range(0, len(res), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(res))
    client.insert("default.LLM_bitcoin_demo_qa_1 ", data[i:i_end+1].to_records(index=True).tolist(), column_names=data.columns.tolist())
    # get metadata
    # meta = batch.to_dict(orient="records")
get_index_status="SELECT count(*) FROM default.LLM_bitcoin_demo_qa_1"
print(f"index build status: {client.query(query=get_index_status)}")

  0%|          | 0/20 [00:00<?, ?it/s]

index build status: 2783


### Finding top k most relevant contexts 

In [29]:
question = 'what is the difference between bitcoin and traditional money?'
emb_query = retriever.encode(question).tolist()

### Filter the result with article_rank < 500

In [30]:
top_k = 10
results = client.query(f"""
SELECT summary, distance('topK={top_k}')(summary_feature, {emb_query}) as dist
FROM default.LLM_bitcoin_demo_qa_1
WHERE article_rank < 500
""")

In [31]:
res = []
for r in results:
    res.append(r[0])

### Get CoT for GPT-3.5

In [32]:
CoT = ''
for i in res:
    CoT += i
CoT += '\n' +'Based on the context above '+'\n' +' Q: '+ question + '\n' +' A: The answer is'

In [33]:
print(CoT)

Bitcoin is the currency of the Internet: a distributed, worldwide, decentralized digital money. Unlike traditional currencies such as dollars, bitcoins are issued and managed without any central authority whatsoever: there is no government, company, or bank in charge of Bitcoin. As such, it is more resistant to wild inflation and corrupt banks. With Bitcoin, you can be your own bank.Bitcoin is the currency of the Internet: a distributed, worldwide, decentralized digital money. Unlike traditional currencies such as dollars, bitcoins are issued and managed without any central authority whatsoever: there is no government, company, or bank in charge of Bitcoin. As such, it is more resistant to wild inflation and corrupt banks. With Bitcoin, you can be your own bank.Bitcoin is the currency of the Internet: a distributed, worldwide, decentralized digital money. Unlike traditional currencies such as dollars, bitcoins are issued and managed without any central authority whatsoever: there is no

### Get result from GPT-3.5

In [35]:
response = openai.Completion.create(
          engine='gpt-3.5-turbo',
          prompt=CoT,
          max_tokens=4000,
          temperature=0,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0,
          stop=["\n"]
        )

In [36]:
print("Example: Retrieval with MyScale")
print('Q: ', question)

print('A: ',response["choices"][0]["text"])

Example: Retrieval with MyScale
Q:  what is the difference between bitcoin and traditional money?
A:  Bitcoin is a decentralized digital currency that is not issued or managed by any central authority, whereas traditional currencies such as dollars are issued and managed by governments or central banks. Bitcoin is also more resistant to inflation and corruption in the banking system, and allows individuals to be their own bank.


We return a complete and detailed answer. We have recieved great results.